In [7]:
import pandas as pd
import numpy as np

# ---------------- TIME DIM ----------------
time_dim = pd.DataFrame({
    "time_key": range(1, 13),
    "day": np.random.randint(1, 29, 12),
    "day_of_the_week": np.random.choice(
        ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], 12),
    "month": range(1, 13),
    "quarter": [(m - 1)//3 + 1 for m in range(1, 13)],
    "year": [2023]*12
})

# Snowflake: separate Year, Quarter, Month
year_dim = pd.DataFrame({"year_key": [1], "year": [2023]})
quarter_dim = pd.DataFrame({
    "quarter_key": [1, 2, 3, 4],
    "quarter": [1, 2, 3, 4],
    "year_key": [1, 1, 1, 1]
})
month_dim = pd.DataFrame({
    "month_key": range(1, 13),
    "month": range(1, 13),
    "quarter_key": [(m - 1)//3+1 for m in range(1, 13)]
})
# Map keys in time_dim
time_dim["month_key"] = time_dim["month"]
time_dim["quarter_key"] = time_dim["quarter"]
time_dim["year_key"] = 1

# ---------------- ITEM DIM ----------------
item_dim = pd.DataFrame({
    "item_key": range(1, 11),
    "item_name": [f"Item_{i}" for i in range(1, 11)],
    "brand": np.random.choice(["BrandA", "BrandB", "BrandC"], 10),
    "type": np.random.choice(["Electronics", "Clothing", "Grocery"], 10),
    "supplier_type": np.random.choice(["Local", "International"], 10)
})

# Snowflake: separate Brand, Type, Supplier
brand_dim = pd.DataFrame({"brand_key": [1, 2, 3], "brand": ["BrandA", "BrandB", "BrandC"]})
type_dim = pd.DataFrame({"type_key": [1, 2, 3], "type": ["Electronics", "Clothing", "Grocery"]})
supplier_dim = pd.DataFrame({"supplier_key": [1, 2], "supplier_type": ["Local", "International"]})

# Map keys in item_dim
brand_lookup = dict(zip(brand_dim["brand"], brand_dim["brand_key"]))
type_lookup = dict(zip(type_dim["type"], type_dim["type_key"]))
supplier_lookup = dict(zip(supplier_dim["supplier_type"], supplier_dim["supplier_key"]))
item_dim["brand_key"] = item_dim["brand"].map(brand_lookup)
item_dim["type_key"] = item_dim["type"].map(type_lookup)
item_dim["supplier_key"] = item_dim["supplier_type"].map(supplier_lookup)

# ---------------- BRANCH DIM ----------------
branch_dim = pd.DataFrame({
    "branch_key": range(1, 6),
    "branch_name": [f"Branch_{i}" for i in range(1, 6)],
    "branch_type": np.random.choice(["Retail", "Wholesale"], 5)
})
branchtype_dim = pd.DataFrame({
    "branch_type_key": [1, 2],
    "branch_type": ["Retail", "Wholesale"]
})
branchtype_lookup = dict(zip(branchtype_dim["branch_type"], branchtype_dim["branch_type_key"]))
branch_dim["branch_type_key"] = branch_dim["branch_type"].map(branchtype_lookup)

# ---------------- LOCATION DIM ----------------
location_dim = pd.DataFrame({
    "location_key": range(1, 6),
    "street": [f"Street_{i}" for i in range(1, 6)],
    "city": np.random.choice(["Mumbai", "Delhi", "Bangalore", "Chennai", "Pune"], 5)
})
city_state_map = {
    "Mumbai": ("Maharashtra", "India"),
    "Pune": ("Maharashtra", "India"),
    "Delhi": ("Delhi", "India"),
    "Bangalore": ("Karnataka", "India"),
    "Chennai": ("Tamil Nadu", "India")
}
city_dim = pd.DataFrame([
    {"city_key": i+1, "city": city,
     "state": city_state_map[city][0], "country": city_state_map[city][1]}
    for i, city in enumerate(city_state_map.keys())
])
city_lookup = dict(zip(city_dim["city"], city_dim["city_key"]))
location_dim["city_key"] = location_dim["city"].map(city_lookup)

state_dim = city_dim[["state", "country"]].drop_duplicates().reset_index(drop=True)
state_dim["state_key"] = state_dim.index+1
state_lookup = dict(zip(state_dim["state"], state_dim["state_key"]))
city_dim["state_key"] = city_dim["state"].map(state_lookup)

country_dim = state_dim[["country"]].drop_duplicates().reset_index(drop=True)
country_dim["country_key"] = country_dim.index+1
country_lookup = dict(zip(country_dim["country"], country_dim["country_key"]))
state_dim["country_key"] = state_dim["country"].map(country_lookup)

# ---------------- FACT TABLE ----------------
fact_sales = pd.DataFrame({
    "time_key": np.random.choice(time_dim["time_key"], 50),
    "item_key": np.random.choice(item_dim["item_key"], 50),
    "branch_key": np.random.choice(branch_dim["branch_key"], 50),
    "location_key": np.random.choice(location_dim["location_key"], 50),
    "units_sold": np.random.randint(1, 100, 50),
    "dollars_sold": np.random.randint(100, 10000, 50),
    "avg_sales": np.random.uniform(50, 500, 50).round(2)
})

# ---------------- SAVE ALL ----------------
time_dim.to_csv("time_dim_snowflake.csv", index=False)
year_dim.to_csv("year_dim.csv", index=False)
quarter_dim.to_csv("quarter_dim.csv", index=False)
month_dim.to_csv("month_dim.csv", index=False)

item_dim.to_csv("item_dim.csv", index=False)
brand_dim.to_csv("brand_dim.csv", index=False)
type_dim.to_csv("type_dim.csv", index=False)
supplier_dim.to_csv("supplier_dim.csv", index=False)

branch_dim.to_csv("branch_dim_snowflake.csv", index=False)
branchtype_dim.to_csv("branchtype_dim.csv", index=False)

location_dim.to_csv("location_dim_snowflake.csv", index=False)
city_dim.to_csv("city_dim.csv", index=False)
state_dim.to_csv("state_dim.csv", index=False)
country_dim.to_csv("country_dim.csv", index=False)

fact_sales.to_csv("fact_sales_snowflake.csv", index=False)
